In [ ]:
# Correr para reiniciar Kernel y la sesion.
#!kill -9 -1

# Proyecto Final (Texto)

### Curso Deep Learning y Redes Neuronales - MIIA4406_01

#### Universidad de los Andes

Integrantes:

|Nombre|Codigo|
|---------------------|---------------------|
|Saby Espinel|*201215868*|
|Diego Salazar|*201628925*| <br>
<br>
______________________________________________________________________________________________________________________

## Libraries

In [1]:
# Verificar que se esta usando GPU
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 1442079591125361260, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 251789312
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 6354184267043998337
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [17]:
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
#from sklearn.multiclass import OneVsRestClassifier
#from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
#from sklearn.metrics import r2_score, roc_auc_score
#from sklearn.model_selection import train_test_split
#from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from skimage import io

# Text processing
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence
from nltk.stem.snowball import SnowballStemmer
from sklearn.model_selection import train_test_split

# Keras NN
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Input, Embedding, LSTM
from keras.optimizers import RMSprop
from keras.callbacks import History
from keras.models import Model
#from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

%matplotlib inline

In [ ]:
##### Para usar en CoLab #####
!pip install -U -q PyDrive
!pip install textblob
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import files


In [ ]:
##### Para usar en CoLab #####
# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Read Data CoLab

In [4]:
##### Para CoLab #####
# Verificacion de contenido de carpetas en el Drive con su respectivo ID
#file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
file_list = drive.ListFile({'q': "'1gX_xAhoYwIwmOVRd33lAuf4pCM3cChB6' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: ExampleTextRF.ipynb, id: 1awW3xWv99qbvcMP1hWTIJjmdmKq8Ku42
title: ExampleImgRF.ipynb, id: 1Pg2-wYIUr866PzKvUy8iXsstZT87uE6x
title: Proyecto 2_birds_DL (1).ipynb, id: 1mRpiAc1wbDI226Zw56g6NvPHK0ebcACn
title: images.zip, id: 1NMIKcqCHJIDaQi4M9mbjrZf0qlpXtrO-
title: images, id: 1pkD2K630_7_sqyt42vnbOYKVZrnyE-VQ
title: Proyecto2.ipynb, id: 131wvpD2jMHusixkRryXoFdTj9DbmbxYp
title: dataTraining.csv, id: 1SKNGe6FFghASG38LjtUaoTrmozUibRCZ
title: dataTesting.csv, id: 1JWWmvheWnofZl2zqY-7Gu2rJprceYB80
title: .ipynb_checkpoints, id: 17KsfZFar28G8dXPBETs6cCpaAknZO3yP
title: moviegenre.png, id: 1h6OU9y8Fwa-HqLMeVDAhLKO9Et6Bojmv
title: README.md, id: 1ICVh_qZvcAXH36lytZyJiYgMFbocGgCd


In [ ]:
##### Para CoLab #####
trainDrive = drive.CreateFile({'id': '1SKNGe6FFghASG38LjtUaoTrmozUibRCZ'})
trainDrive.GetContentFile('dataTraining.csv')
dataTraining=pd.read_csv("dataTraining.csv", index_col=0)

testDrive = drive.CreateFile({'id': '1JWWmvheWnofZl2zqY-7Gu2rJprceYB80'})
testDrive.GetContentFile('dataTesting.csv')
dataTesting=pd.read_csv("dataTesting.csv", index_col=0)

# Read Data Jupyter

In [34]:
##### Para CoLab #####
# Lectura de archivos en Jupyter desde el PC
path = 'C:/Users/da.salazarb/Google Drive (dasalazarb@unal.edu.co)/AppliedDeepLearningClass-master/finalProject'
dataTraining = pd.read_csv(os.path.join(path, 'dataTraining.csv'), encoding='UTF-8', index_col=0)
dataTesting = pd.read_csv(os.path.join(path, 'dataTesting.csv'), encoding='UTF-8', index_col=0)
#print(dataTraining.head())
#print(dataTesting.head())

## Cleaning data (dataTraining['post'] and dataTesting['post'])

In [5]:
# Definicion de funcion para preprocesar el texto
def preProssText(df, commonWord, numberCommonWords, rareWord, numberRareWords, spellCorrect):
  # Remove punctutation
  df['plot'] = df['plot'].str.replace('[^\w\s]','')
  
  # Lower case
  df['plot'] = df['plot'].apply(lambda x: " ".join(x.lower() for x in x.split()))
  
  # Remove Common words
  if commonWord == True:
    freq = pd.Series(' '.join(df['plot']).split()).value_counts()[:numberCommonWords]
    df['plot'] = df['plot'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
  
  # Remove Rare words
  if rareWord == True:
    freq = pd.Series(' '.join(df['plot']).split()).value_counts()[-numberRareWords:]
    df['plot'] = df['plot'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
  
  # Spelling correction -> Tener cuidado puede cambiar los valores. Ej: "ur" por "or" en vez de "your"
  if spellCorrect == True:
    from textblob import TextBlob
    df['plot'] = df['plot'].apply(lambda x: str(TextBlob(x).correct()))
    
  return df

In [35]:
# Pre-procesamiento de texto
dataTraining = preProssText(dataTraining, commonWord=True, numberCommonWords=10, rareWord=True, numberRareWords=10, spellCorrect=False)
dataTesting = preProssText(dataTesting, commonWord=True, numberCommonWords=10, rareWord=True, numberRareWords=10, spellCorrect=False)

In [36]:
dataTraining.head()

,year,title,plot,genres,rating
3107,2003,Most,most story single father who takes eight year ...,"['Short', 'Drama']",8.0
900,2008,How to Be a Serial Killer,serial killer decides teach secrets satisfying...,"['Comedy', 'Crime', 'Horror']",5.6
6724,1941,A Woman's Face,sweden female blackmailer with disfiguring fac...,"['Drama', 'Film-Noir', 'Thriller']",7.2
4704,1954,Executive Suite,friday afternoon new york president tredway co...,['Drama'],7.4
2582,1990,Narrow Margin,los angeles editor publishing house carol hunn...,"['Action', 'Crime', 'Thriller']",6.6


In [37]:
dataTesting.head()

,year,title,plot
1,1999,Message in a Bottle,who meets by fate shall be sealed by fate ther...
4,1978,Midnight Express,true story billy hayes an american college stu...
5,1996,Primal Fear,martin vail left chicago da office become succ...
6,1950,Crisis,husband wife americans dr eugene mrs helen fer...
7,1959,The Tingler,coroner scientist dr warren chapin researching...


***************************************************************************************************

# Create count vectorizer with ngrams

In [9]:
# define a function that accepts text and returns a list of lemmas
import nltk
nltk.download('wordnet')

def split_into_lemmas(text):
    from nltk.stem import WordNetLemmatizer
    wordnet_lemmatizer = WordNetLemmatizer()
    
    text = text.lower()
    words = text.split()
    return [wordnet_lemmatizer.lemmatize(word) for word in words]

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\da.salazarb\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [10]:
# define a function that accepts text and returns a list of stem
def split_into_stem(text):
    stemmer = SnowballStemmer('english')
    text = text.lower()
    words = text.split()
    return [stemmer.stem(word) for word in words]

In [58]:
vect = CountVectorizer(analyzer=split_into_lemmas, ngram_range=(1, 1), max_features=1000, stop_words='english')
X_dtm = vect.fit_transform(dataTraining['plot'])
X_dtm.shape

(7895, 1000)

In [12]:
print(vect.get_feature_names()[:50])

['abandon', 'abl', 'about', 'abus', 'accept', 'accid', 'accident', 'across', 'act', 'action', 'activ', 'actor', 'actress', 'actual', 'adam', 'adult', 'adventur', 'affair', 'affect', 'after', 'again', 'against', 'age', 'agent', 'ago', 'agre', 'aid', 'air', 'alcohol', 'alex', 'alien', 'aliv', 'all', 'allow', 'almost', 'alon', 'along', 'alreadi', 'also', 'although', 'alway', 'america', 'american', 'among', 'an', 'angel', 'ani', 'anim', 'ann', 'anna']


***************************************************************************************************

# Create y

In [13]:
dataTraining.head()

,year,title,plot,genres,rating
3107,2003,Most,most story single father who takes eight year ...,"['Short', 'Drama']",8.0
900,2008,How to Be a Serial Killer,serial killer decides teach secrets satisfying...,"['Comedy', 'Crime', 'Horror']",5.6
6724,1941,A Woman's Face,sweden female blackmailer with disfiguring fac...,"['Drama', 'Film-Noir', 'Thriller']",7.2
4704,1954,Executive Suite,friday afternoon new york president tredway co...,['Drama'],7.4
2582,1990,Narrow Margin,los angeles editor publishing house carol hunn...,"['Action', 'Crime', 'Thriller']",6.6


In [14]:
dataTraining['genres'] = dataTraining['genres'].map(lambda x: eval(x))

le = MultiLabelBinarizer()
y_genres = le.fit_transform(dataTraining['genres'])

In [15]:
y_genres.shape

(7895, 24)

******************************************************************************************************************************************

# Split train and test

In [61]:
X_train, X_test, y_train_genres, y_test_genres = train_test_split(X_dtm, y_genres, test_size=0.33, random_state=42)

# Train DeepNeuralNetwork

In [59]:
# Modelo
top_words = 1000
max_review_length = 1000
# Set the model
def textModel(max_review_length):
    model = Sequential()
    model.add(Embedding(top_words, 32, input_length=max_review_length))
    #model.add(BatchNormalization())
    model.add(LSTM(256, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(LSTM(100))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(24, activation='sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0), metrics=['accuracy'])
    return model

In [62]:
ix_train=np.random.choice(X_train.shape[0], 1000)
ix_test=np.random.choice(X_test.shape[0], 100)
model=textModel(max_review_length)
model.fit(X_train[ix_train], y_train_genres[ix_train], epochs=3, batch_size=10, validation_data=(X_test[ix_test], y_test_genres[ix_test]))
#model.fit(X_dtm, y_genres, epochs=3, batch_size=10)
# epochs=5, batch_size=5 -> acc: 0.18

Train on 300 samples, validate on 100 samples
Epoch 1/3
300/300 [==============================] - 63s 209ms/step - loss: 9.2630 - acc: 0.0600 - val_loss: 11.4861 - val_acc: 0.0000e+00
Epoch 2/3
300/300 [==============================] - 60s 200ms/step - loss: 9.0080 - acc: 0.1267 - val_loss: 10.7203 - val_acc: 0.0100
Epoch 3/3
300/300 [==============================] - 59s 197ms/step - loss: 8.7869 - acc: 0.1400 - val_loss: 13.1403 - val_acc: 0.0600


******************************************************************************************************************************************

******************************************************************************************************************************************

# Images

## Libraries

In [ ]:
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
#from sklearn.multiclass import OneVsRestClassifier
#from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
#from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import train_test_split
#from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from skimage import io

## Image transformation and reading
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import cv2
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.layers import Dense, Activation, Dropout, Input
from keras.models import Model

%matplotlib inline

In [ ]:
##### Solo para Colab #####
!pip install -U -q PyDrive
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import files

In [ ]:
# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# Tome las primeras 1025 + 1055 imagenes
!unzip -o images.zip

In [ ]:
path='images/'
#image = io.imread(os.path.join(path, 'images_resize_gray', str(dataTraining.index[0]) + '_resize_gray.jpeg'))
image = io.imread(os.path.join(path, '1713.jpeg'))

******************************************************************************************************************************************

# Read Images CoLab

In [ ]:
# Verificacion de contenido de carpetas en el Drive con su respectivo ID
#file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
file_list = drive.ListFile({'q': "'1gX_xAhoYwIwmOVRd33lAuf4pCM3cChB6' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

In [ ]:
# Download a file based on its file ID.
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1NMIKcqCHJIDaQi4M9mbjrZf0qlpXtrO-' # -> cnn_data.zip
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile(downloaded['title'])

In [ ]:
desired_size=200
imgFinal = np.ones((1,200,200,3))
for i in os.listdir('images'):
  if i.startswith('desktop'):
    pass
  else:
    #images_training.append(load_img(os.path.join(path, str(i)),grayscale=False,target_size=None,interpolation='nearest'))
    #image=load_img(os.path.join(path, str(i)),grayscale=False,target_size=None,interpolation='nearest')
    #image=img_to_array(image)
    
    im = cv2.imread(os.path.join(path, str(i)))
    old_size = im.shape[:2] # old_size is in (height, width) format

    ratio = float(desired_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])

    # new_size should be in (width, height) format

    im = cv2.resize(im, (new_size[1], new_size[0]))

    delta_w = desired_size - new_size[1]
    delta_h = desired_size - new_size[0]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)

    color = [0, 0, 0]
    new_im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT,value=color)
    new_im=new_im.reshape((1,new_im.shape[0],new_im.shape[1],new_im.shape[2]))
    imgFinal = np.concatenate([imgFinal, new_im], axis=0)

# Code for padding: https://jdhao.github.io/2017/11/06/resize-image-to-square-with-padding/

In [ ]:
#imgFinal_2=np.rollaxis(imgFinal, 3, 1) #Para cuando le de la locura al ImageDataGenerator
imgFinal=np.delete(imgFinal, 0, 0)
imgFinal.shape
datagen = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True)
imgFinal_2=imgFinal.reshape((imgFinal.shape[0],200,200,3))
datagen.fit(imgFinal_2)

# Create *'y'* Colab

In [ ]:
# Verificacion de contenido de carpetas en el Drive con su respectivo ID
#file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
file_list = drive.ListFile({'q': "'1SKNGe6FFghASG38LjtUaoTrmozUibRCZ' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

In [ ]:
trainDrive = drive.CreateFile({'id': '1SKNGe6FFghASG38LjtUaoTrmozUibRCZ'})
trainDrive.GetContentFile('dataTraining.csv')
dataTraining=pd.read_csv("dataTraining.csv", index_col=0)

testDrive = drive.CreateFile({'id': '1JWWmvheWnofZl2zqY-7Gu2rJprceYB80'})
testDrive.GetContentFile('dataTesting.csv')
dataTesting=pd.read_csv("dataTesting.csv", index_col=0)

In [ ]:
ix_movies = []
for i in range(0,len(os.listdir("images/"))):
  if os.listdir("images/")[i] == "desktop.ini":
    pass
  else:
    ix_movies.append(int(os.listdir("images/")[i].replace(".jpeg","")))

In [ ]:
imgTrain=[]
imgTest=[]
data_training=[]
data_testing=[]
for i in range(0,len(ix_movies)):
  try:
    data_training.append(dataTraining.loc[ix_movies[i]])
    imgTrain.append(i)
  except:
    data_testing.append(dataTesting.loc[ix_movies[i]])
    imgTest.append(i)

#ix_movies[2]


#print(dataTraining.iloc[4082])#.index#.loc[ix_movies, :]
#print(dataTraining.iloc[ix_movies[2]])

In [22]:
len(imgTrain)

1432

In [ ]:
imgTraining=imgFinal_2[[imgTrain[:1431]]]

In [22]:
# Son 714 que vienen del dataTraining, es decir, tienen labels definidos y son los
#deben ser usados para seprar en train y test
print(imgTraining.shape)
print(np.array(data_training).shape)

(1432, 224, 224, 3)
(1432, 5)


In [17]:
data_training=pd.DataFrame(data_training)
data_training.head()

,year,title,plot,genres,rating
9873,1981,They All Laughed,new york ' s odyssey detective agency is hired...,"['Comedy', 'Romance']",6.4
856,1958,A Time to Love and a Time to Die,"in N , a company of german soldiers on the ...","['Drama', 'Romance', 'War']",7.8
4384,2001,The Invisible Circus,"in N , at age N , phoebe is unhappy . w...",['Drama'],5.6
2405,1976,Eaten Alive,judd runs the starlight hotel out in some sort...,"['Horror', 'Thriller']",5.4
8005,1983,The Sting II,lonnegan is planning to get back at gondorff a...,"['Comedy', 'Crime']",5.0


In [ ]:
data_training['genres'] = data_training['genres'].map(lambda x: eval(x))

le = MultiLabelBinarizer()
y_genres = le.fit_transform(data_training['genres'])

In [ ]:
imgTesting=imgFinal_2[[imgTest]] #Esto se pueden obviar
imgTesting.shape

In [27]:
y_genres.shape

(1432, 23)

In [ ]:
X_train, X_test, y_train_genres, y_test_genres = train_test_split(imgTraining, y_genres[:1431], test_size=0.33, random_state=42)

In [26]:
print(X_train.shape)
print(y_test_genres.shape)

(959, 224, 224, 3)
(473, 23)


# Read Images Jupyter

In [ ]:
path = ''
dataTraining = pd.read_csv(os.path.join(path, 'data', 'dataTraining.csv'), encoding='UTF-8', index_col=0)
dataTesting = pd.read_csv(os.path.join(path, 'data', 'dataTesting.csv'), encoding='UTF-8', index_col=0)

In [ ]:
# Training
images_training = []
for i in tqdm(dataTraining.index):
    images_training.append(io.imread(os.path.join(path, 'images_resize_gray', str(i) + '_resize_gray.jpeg')).flatten())

In [ ]:
images_training = np.stack(images_training)

In [ ]:
# Testing
images_testingimages_t  = []
for i in tqdm(dataTesting.index):
    images_testing.append(io.imread(os.path.join(path, 'images_resize_gray', str(i) + '_resize_gray.jpeg')).flatten())

images_testing = np.stack(images_testing)

images_testing.shape, dataTesting.shape[0]

In [ ]:
pcapca  ==  PCAPCA((n_componentsn_compon =32)
images_training_pca = pca.fit_transform(images_training)

In [ ]:
images_testing_pca = pca.transform(images_testing)

# Create *'y'* Jupyter

# Apply models to kaggle test

In [ ]:
X_test_dtm = vect.transform(dataTesting['plot'])

cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

y_pred_test_genres = model.predict_proba(X_test_dtm)

pd.DataFrame(y_pred_test_genres, index=dataTesting.index, columns=cols).to_csv('pred_genres_text_RF.csv', index_label='ID')